# output metric

In [15]:
output_metric = {
                "Total CO2e Emissions":
                    ["Emissions?"],
                "CO2e Emissions, Scope 1":
                    ["Emissions?", "Scope 1"],
                "CO2e Emissions, Scope 2":
                    ["Emissions?", "Scope 2"],
                "CO2e Emissions, Scope 3":
                    ["Emissions?", "Scope 3"],
                "Emissions Policy":
                    ["Emissions?"],
                "Emission Reduction Target %":
                    ["Emissions?"],
                "Emissions Reduction Target Year":
                    ["Emissions?"],
                "Total Water Withdrawal":
                    ["Water"],
                "Water Discharged":
                    ["Water"],
                "Total Fresh Water Withdrawal":
                    ["Water"],
                "Water Recycled":
                    ["Water"],
                "Water Efficiency Target":
                    ["Water"],
                "Total Energy Consumed":
                    ["Energy"],
                "Total Renewable Energy":
                    ["Energy"],
                "Renewable Energy Consumed":
                    ["Energy"],
                "Renewable Energy Use":
                    ["Energy"],
                "Total Waste":
                    ["Waste"],
                "Women Executives %":
                    ["Women", "Female"],
                "Women Board Members %":
                    ["Women", "Female"],
                "Total Board Members":
                    ["Board Members?"],
                "Independent Board Members %":
                    ["Board Members?"],
                "Audit Committee Independence %":
                    ["Committee Independence"],
                "Compensation Committee Independence %":
                    ["Committee Independence"],
                "Nomination Committee Independence %":
                    ["Committee Independence"],
                "ESG Sustainability Reporting":
                    ["Sustainability Reporting", "ESG reporting"],
                "Lost Time Injury Rate - Employees":
                    ["Injury", "Emlployees?"],
                "Avg Training Hours / Employee":
                    ["Training Hours?", "Employee"],
                "Trade Union Percentage":
                    ["Trade Union"],
                "CEO Salary / Average Salary":
                    ["Salary", "CEO Salary", "Average Salary"],
                "Employee Turnover %":
                    ["Employee", "Turnover"],
                "Involuntary Employee Turnover Rate":
                    ["Employee", "Turnover"],
                "Women's / Men's Compensation %":
                    ["Women", "Female", "Men", "Male"],
                "Women Employees %":
                    ["Women", "Female", "Employees?"],
                "Women Managers %":
                    ["Women", "Female", "Managers?"],
                "Minority Employees %":
                    ["Minority", "Employees?"]
                }

In [16]:
# output_metric= {
#                 "Emissions":
#                     ["emission", "co2 emission", "ghg emission", "emission reduction", "emission target", "Emissions Policy"],
#                 "Water":
#                     ["Water Withdrawal", "Water Discharged", "Water Recycled", "water risk"],
#                 "Energy":
#                     ["Energy Consumed", "Renewable Energy"],
#                 "Business Ethics":
#                     ["Women Executives", "Women Board Members", "Board Members", "Committee Independence", "ESG Sustainability Reporting"],
#                 "Labor Practices":
#                     ["Trade Union", "CEO Salary", "Average Salary", "Employee Turnover", "Avg Training Hours"],
#                 "Employee Engagement, Diversity & Inclusion":
#                     ["Women Employees", "Women Managers", "Minority Employees"],
#                 "Employee Health & Safety":
#                     ["Lost Time", "Injury Rate"],
#                 "Waste":
#                     ["Waste"]
#                 }

# tables

In [17]:
import camelot
import os

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i : i + n]
def get_chunks(filepath, pages, chunk=10):
    """
    Divide the extraction work into n chunks and return this chunks.

    filepath : str
        Filepath or URL of the PDF file.
    pages : str, optional (default: '1')
        Comma-separated page numbers.
        Example: '1,3,4' or '1,4-end' or 'all'.
    """

    # get list of pages from camelot.handlers.PDFHandler
    handler = camelot.handlers.PDFHandler(filepath)
    page_list = handler._get_pages(pages=pages)
    # chunk pages list
    page_chunks = list(chunks(page_list, chunk))

    return page_chunks

In [19]:
def check_information_table(tables):
    for table in tables:
        # print(table.cols)
        # print(table.rows)
        # print(tabel.accuracy)
        table_df = table.df
        columns = table_df.shape[1]
        for key_word,value_words  in output_metric.items():
            for col in range(columns):
                contains = any(table.df[col].str.contains("|".join(value_words), case=False, regex=True).tolist())
                if contains:
                    position_record_table[key_word].append(str(table.page))
                    break

In [20]:
def parseTable_camelot(input_path, file, output_path, pages = "all"):
    print("parsing table from " + file + " at "+ pages + " pages " )
    try:
        filepath = os.path.join(input_path, file)
        page_chunks = get_chunks(filepath, pages=pages)
        for chunk in page_chunks:
            pages_string = str(chunk).replace("[", "").replace("]", "")
            tables = camelot.read_pdf(filepath, pages=pages_string)
            if len(tables) > 0:
                check_information_table(tables)
                # print("saving {len(tables)} tables")
                # accuracy = [str(table.parsing_report["accuracy"]) for table in tables]
                # print("accuracy list " + " ".join(accuracy))
                # tables.export(os.path.join(output_path, file.replace("pdf", "xlsx")), f='excel')
            else:
                print(f"no tables found for " + file)
    except Exception as e:
        print("error parsing table from " + file)    
        print(e)      

In [21]:
import tabula
import pandas as pd
def parseTable_tabula(input_path, file, output_path, pages):
    print("parsing table from " + file + "at pages" + pages)
    try:
        tables = tabula.read_pdf(os.path.join(input_path, file), pages=pages, lattice=True)
        if len(tables) > 0:
            print("saving {len(tables)} tables")
            with pd.ExcelWriter(os.path.join(output_path, file.replace("pdf", "xlsx"))) as f_obj:
                for i,table in enumerate(tables):
                    table.to_excel(f_obj, sheet_name=str(i))
        else:
            print(f"no tables found for " + file)
    except Exception as e:
        print("error parsing table from " + file)    
        print(e) 


# texts

In [22]:
import re
def check_information_text(page, text):
    for key_word, value_word in output_metric.items():
        pattern = "|".join(value_word)
        rex = re.search(pattern, text, re.I)
        if rex != None:
            position_record_text[key_word].append(str(page))

In [23]:
import PyPDF2
def parseText(input_path, file, output_path, pages):
    print("parsing text from " + file + " at " + pages + " pages ")
    object = PyPDF2.PdfFileReader(os.path.join(input_path, file))
    num_pages = object.getNumPages()
    for i in range(0, num_pages):
        page = object.getPage(i)
        text = page.extractText()
        text = text.replace('\n','')
        check_information_text(i, text)
        # with open(os.path.join(output_path, file.replace("pdf", "text")), "a") as f_obj:
        #     f_obj.write(text)

# TF-IDF

In [24]:
import nltk
import math
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *

In [25]:
def get_tokens(text):
    lower = text.lower()
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    no_punctuation = lower.translate(remove_punctuation_map)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [26]:
def tf(word, count):
    return count[word] / sum(count.values())
def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)
def idf(word, count_list):
    return math.log(len(count_list)) / (1 + n_containing(word, count_list))
def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)

In [27]:
def count_term(text):
    tokens = get_tokens(text)
    filtered = [w for w in tokens if not w in stopwords.words('english')]
    stemmer = PorterStemmer()
    stemmed = stem_tokens(filtered, stemmer)
    count = Counter(stemmed)
    return count

def cal_tfidf(text_list, key_words):
    countlist = []
    for text in text_list:
        countlist.append(count_term(text))
    for i, count in enumerate(countlist):
        print("Top words in document {}".format(i + 1))
        scores = {word: tfidf(word, count, countlist) for word in count}
        sorted_words = sorted(scores.items(), key = lambda x: x[1], reverse=True)
        for word, score in sorted_words[:5]:
            print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

# main function

In [28]:
import os
import pandas as pd

input_path = "./pdf_files"
output_table_camelot = "./table_files_camelot"
output_table_tabula = "./table_files_tabula"
output_path_text = "./text_files"
pdf_files = os.listdir(input_path)
name = ["output_metric", "page"]
for file in pdf_files:
    # 1 parsing the table and text
    position_record_table = {key_word:[] for key_word in output_metric.keys()}
    position_record_text = {key_word:[] for key_word in output_metric.keys()}
    parseTable_camelot(input_path, file, output_table_camelot, pages="all")
    # parseTable_tabula(input_path, file, output_table_tabula)
    parseText(input_path, file, output_path_text, "all")
    # 2 save the result
    position_record = {key_word:position_record_table[key_word]+position_record_text[key_word] 
                       for key_word in output_metric.keys()}
    position_record = {key_word:list(set(pages)) for key_word, pages in position_record.items()}
    position_matchs = [[key, " ".join(value)] for key, value in position_record.items()]
    data = pd.DataFrame(columns=name, data=position_matchs)
    data.to_csv("./data_extraction/" + file.replace(".pdf", "_hybird.csv"))

parsing table from 2021-tesla-impact-report.pdf at pages all
check the information
check the information
check the information
check the information
no tables found for 2021-tesla-impact-report.pdf
check the information
check the information
check the information
check the information
check the information
check the information
check the information
check the information


/root/miniconda3/envs/camelotenv/lib/python3.10/site-packages/camelot/parsers/lattice.py:411: UserWarning: page-138 is image-based, camelot only works on text-based pages.
  warnings.warn(


check the information
no tables found for 2021-tesla-impact-report.pdf
parsing text from 2021-tesla-impact-report.pdf at pages all
